<a href="https://colab.research.google.com/github/kellischeuble/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module3-join-and-reshape-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [1]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2020-01-17 00:05:13--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.100.205
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.100.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  41.1MB/s    in 5.1s    

2020-01-17 00:05:18 (38.6 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [2]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [3]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [4]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [5]:
# Look for products.. I'm guessing the info we need is in orders and products

import pandas as pd

orders = pd.read_csv('orders.csv')
orders.head()


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [6]:
# Not exactly what I'm looking for...
# Let's try products

products = pd.read_csv('products.csv')
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [7]:
# Look at order_products__train first because I know that file is smaller

order_products__train = pd.read_csv('order_products__train.csv')
order_products__train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [8]:
# OK, so know I know that I want to combine the three dataframes
# products, order_products__prior, and order_products__train
# The columns I need are the product_id and product name

# So, if I can find the product ID for each (in the product list), I can then
# find the number of times that they are ordered when i look at the 
# order_products lists

# create condition
top_ten = ['Banana',
          'Bag of Organic Bananas',
          'Organic Strawberries',
          'Organic Baby Spinach',
          'Organic Hass Avocado',
          'Organic Avocado',
          'Large Lemon',
          'Strawberries',
          'Limes',
          'Organic Whole Milk'
          ]

frequent_products = products[products['product_name'].isin(top_ten)]
frequent_products.head(10)

,product_id,product_name,aisle_id,department_id
13175,13176,Bag of Organic Bananas,24,4
16796,16797,Strawberries,24,4
21136,21137,Organic Strawberries,24,4
21902,21903,Organic Baby Spinach,123,4
24851,24852,Banana,24,4
26208,26209,Limes,24,4
27844,27845,Organic Whole Milk,84,16
47208,47209,Organic Hass Avocado,24,4
47625,47626,Large Lemon,24,4
47765,47766,Organic Avocado,24,4


In [9]:
# only need product_id column and the product_name columns.. drop others

frequent_products = frequent_products[['product_name', 'product_id']]
frequent_products

,product_name,product_id
13175,Bag of Organic Bananas,13176
16796,Strawberries,16797
21136,Organic Strawberries,21137
21902,Organic Baby Spinach,21903
24851,Banana,24852
26208,Limes,26209
27844,Organic Whole Milk,27845
47208,Organic Hass Avocado,47209
47625,Large Lemon,47626
47765,Organic Avocado,47766


In [10]:
# Now merge order_products_prior and order_products_train..
# first clean it a bit.. get rid of columns we don't need

order_products__train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [11]:
# I think we only care about product_id because each row 
# is another order... but to be safe I'm going to make another
# dataframe instead of writing over

product_id_train = order_products__train['product_id']
train_len = len(product_id_train)
product_id_train.head(20)

0     49302
1     11109
2     10246
3     49683
4     43633
5     13176
6     47209
7     22035
8     39612
9     19660
10    49235
11    43086
12    46620
13    34497
14    48679
15    46979
16    11913
17    18159
18     4461
19    21616
Name: product_id, dtype: int64

In [12]:
# do the same for order_products__prior

order_products__prior = pd.read_csv('order_products__prior.csv')
order_products__prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [13]:
product_id_prior = order_products__prior['product_id']
prior_len = len(product_id_prior)
product_id_prior.head()

0    33120
1    28985
2     9327
3    45918
4    30035
Name: product_id, dtype: int64

In [14]:
# concatenate by rows

product_id = pd.concat([product_id_train, product_id_prior])
product_id.head()

0    49302
1    11109
2    10246
3    49683
4    43633
Name: product_id, dtype: int64

In [15]:
# check to see if values add up correctly
len(product_id) == prior_len + train_len

True

In [16]:
# Woo! now we merge...

final = pd.merge(product_id,
                       frequent_products,
                       on='product_id')

final.head(20)

,product_id,product_name
0,13176,Bag of Organic Bananas
1,13176,Bag of Organic Bananas
2,13176,Bag of Organic Bananas
3,13176,Bag of Organic Bananas
4,13176,Bag of Organic Bananas
5,13176,Bag of Organic Bananas
6,13176,Bag of Organic Bananas
7,13176,Bag of Organic Bananas
8,13176,Bag of Organic Bananas
9,13176,Bag of Organic Bananas


In [17]:
# YAYYYYY looks like we have it..

final['product_name'].value_counts()

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64

## Reshape Data Section

- Replicate the lesson code. Remember, if you haven't followed along typing out what we did during lecture, do that now to get more repetition with the syntax.
- Take table 2 (the transpose of table 1) and practice changing it into Tidy format and back again. You should not use the transpose operation anywhere in this code:
 - Table 2 --> Tidy
 - Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [18]:
import pandas as pd
import numpy as np

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = table1.T

table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [19]:
# table2 -> tidy
# reset index

table2 = table2.reset_index()
table2.head()

,index,John Smith,Jane Doe,Mary Johnson
0,treatmenta,NaN,16.0,3.0
1,treatmentb,2.0,11.0,1.0


In [20]:
# tidy -> table2

tidy = table2.melt(id_vars='index', 
                   value_vars=['John Smith', 
                               'Jane Doe', 
                               'Mary Johnson'],
                  )
tidy

,index,variable,value
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [21]:
# rename

tidy = tidy.rename(columns={
    'index': 'treatment',
    'variable': 'name',
    'value': 'result'
})

tidy

,treatment,name,result
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [22]:
# change treatment values to 'a' or 'b'

tidy['treatment'] = tidy['treatment'].str.replace('treatment', '')
tidy

,treatment,name,result
0,a,John Smith,NaN
1,b,John Smith,2.0
2,a,Jane Doe,16.0
3,b,Jane Doe,11.0
4,a,Mary Johnson,3.0
5,b,Mary Johnson,1.0


In [23]:
# now go back to wide format

table2 = tidy.pivot_table(index='treatment', columns='name', values='result')
table2.reset_index()

name,treatment,Jane Doe,John Smith,Mary Johnson
0,a,16.0,NaN,3.0
1,b,11.0,2.0,1.0


In [24]:
table2 = table2.reset_index().rename_axis('',axis=1)
table2

,treatment,Jane Doe,John Smith,Mary Johnson
0,a,16.0,NaN,3.0
1,b,11.0,2.0,1.0


In [0]:


import seaborn as sns
flights = sns.load_dataset('flights')

In [26]:
flights.head()

,year,month,passengers
0,1949,January,112
1,1949,February,118
2,1949,March,132
3,1949,April,129
4,1949,May,121


In [27]:
# Flights Pivot Table
# Load seaborn's flights dataset by running the cell below. Then create a pivot 
# table showing the number of passengers by month and year. Use year for the 
# index and month for the columns. You've done it right if you get 112 passengers
# for January 1949 and 432 passengers for December 1960.

flights.pivot_table('passengers', index='year', columns='month')

month,January,February,March,April,May,June,July,August,September,October,November,December
year,,,,,,,,,,,,
1949,112,118,132,129,121,135,148,148,136,119,104,118
1950,115,126,141,135,125,149,170,170,158,133,114,140
1951,145,150,178,163,172,178,199,199,184,162,146,166
1952,171,180,193,181,183,218,230,242,209,191,172,194
1953,196,196,236,235,229,243,264,272,237,211,180,201
1954,204,188,235,227,234,264,302,293,259,229,203,229
1955,242,233,267,269,270,315,364,347,312,274,237,278
1956,284,277,317,313,318,374,413,405,355,306,271,306
1957,315,301,356,348,355,422,465,467,404,347,305,336


## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [28]:
# make list of top 25 latest ordered popular products
# from looking at the graph, it looks like the late foods spike after hour 15
# so, what I want to do is look at all of the foods between time frame 15-19
# then, tally up foods and get top 25

orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [29]:
late = (orders['order_hour_of_day'] >= 15) & (orders['order_hour_of_day'] <=18)

late_orders = orders[late]
print((late_orders).shape)
late_orders.head()

(967899, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
4,431534,1,prior,5,4,15,28.0
8,2295261,1,prior,9,1,16,0.0
18,788338,2,prior,8,1,15,27.0
28,3002854,3,prior,3,3,16,21.0
29,2037211,3,prior,4,2,18,20.0


In [30]:
# I think we only need order_id and order_hour_of_day...

late_orders_subset = late_orders[['order_id', 'order_hour_of_day']]
print((late_orders_subset).shape)
late_orders_subset.head(20)

(967899, 2)


,order_id,order_hour_of_day
4,431534,15
8,2295261,16
18,788338,15
28,3002854,16
29,2037211,18
30,2710558,17
31,1972919,16
32,1839752,15
33,3225766,17
34,3160850,16


In [31]:
# merge order_products__train and __prior
# want to keep the order_id and product_id

order_products__train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [32]:
order_prod_train = order_products__train[['order_id', 'product_id']]
print(order_prod_train.shape)
order_prod_train.head()

(1384617, 2)


,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633


In [33]:
# do same with __prior

order_prod_prior = order_products__prior[['order_id', 'product_id']]
print(order_prod_prior.shape)
order_prod_train.head()

(32434489, 2)


,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633


In [34]:
# concat order_prod_train and order_prod_prior

order_prod = pd.concat([order_prod_train, order_prod_prior])
order_prod.shape

(33819106, 2)

In [35]:
# want to merge based on order_id

merged_late_orders = pd.merge(order_prod, late_orders_subset, on='order_id')
merged_late_orders.head()

,order_id,product_id,order_hour_of_day
0,36,39612,18
1,36,19660,18
2,36,49235,18
3,36,43086,18
4,36,46620,18


In [36]:
# need to add the product_name
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [37]:
products_subset = products[['product_id', 'product_name']]
print(products_subset.shape)
products_subset.head()

(49688, 2)


,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [38]:
# merge products_subset and merged_late_ordes on product_id
all_late_products = pd.merge(merged_late_orders, products_subset, on='product_id')
print(all_late_products.shape)
all_late_products.head()

(9329280, 4)


,order_id,product_id,order_hour_of_day,product_name
0,36,39612,18,Grated Pecorino Romano Cheese
1,5466,39612,18,Grated Pecorino Romano Cheese
2,321948,39612,15,Grated Pecorino Romano Cheese
3,496658,39612,15,Grated Pecorino Romano Cheese
4,1015135,39612,16,Grated Pecorino Romano Cheese


In [39]:
# now we just have to count up product_name

all_late_products['product_name'].value_counts()

Banana                                           135783
Bag of Organic Bananas                           107250
Organic Strawberries                              74876
Organic Baby Spinach                              69274
Organic Hass Avocado                              62108
                                                  ...  
Kings Hawaiian Smoked Bacon Bbq Sauce                 1
Clearly Sheer Beach/Pool Lotion Spray SPF 50+         1
Spicy Peanut Chicken & Broccolli                      1
Breton White Bean Crackers with Salt & Pepper         1
Orange Multihealth Fiber Packets                      1
Name: product_name, Length: 48348, dtype: int64

In [40]:
# but we really only care about the ones that are greater than 2900
# create condition

popular = ((all_late_products['product_name'].value_counts()) >= 2900)

final = all_late_products[popular]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


IndexingError: ignored

In [49]:
# let's try to just sort the values
pd.set_option("display.max_rows", 30)
later = all_late_products.sort_values(by='product_id', ascending= False, axis=1)

later.head()

KeyError: ignored

In [45]:
all_late_products.head()

,order_id,product_id,order_hour_of_day,product_name
0,36,39612,18,Grated Pecorino Romano Cheese
1,5466,39612,18,Grated Pecorino Romano Cheese
2,321948,39612,15,Grated Pecorino Romano Cheese
3,496658,39612,15,Grated Pecorino Romano Cheese
4,1015135,39612,16,Grated Pecorino Romano Cheese


## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####